# ISOLATION FOREST FOR ANOMALY DETECTION

Algoritmos de Isolation Forest para detecção de Anolamia na Folha de Pagamento
Importação do Modulo de Pré-processamento

In [4]:
import sys
sys.path.append("C:/Users/joaoc/Documents/MT/pagamento_servidores/src")
import process_servants as ps
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
import pandas as pd
from janitor import clean_names

## Função para pre-processamento dos dados
Utiliza-se funções importadas do arquivo process_servants.py, no qual foram pré-definidas algumas funções para o pré-processamento dos dados

In [6]:
def process_registration_data_rendim(file_path):
    """Process registration data from Excel file"""
    
    df = (
        pd.read_excel(file_path)
        .pipe(clean_names)
        .pipe(ps.convert_rendim_to_numeric)
        .pipe(ps.process_month_column)
        .pipe(ps.padronizar_ferias)
        .pipe(ps.padronizar_decimo_terceiro)
        )
    df = df.groupby(['cpf_servidor', 'mes', 'rubrica'])['rendim'].sum().reset_index()
    df = df.pivot(index = ['cpf_servidor', 'mes'], columns='rubrica', values='rendim').reset_index()

    return df

## Função para Leitura dos Dados

In [8]:
def processed_data(brute_data_file_path, file_name): 

    registration_df = process_registration_data_rendim(brute_data_file_path)
    
    processed_path  = "C:/Users/joaoc/Documents/MT/pagamento_servidores/data/processed/" + file_name + ".xlsx"
    print("Processed data shapes:")
    print(f"Registration data: {registration_df.shape}")
    
    #registration_df.to_excel(processed_path, index=False)
    
    return registration_df

Importação da Base de Dados

In [10]:
brute_file_path = 'C:/Users/joaoc/Documents/MT/pagamento_servidores/data/raw/servidores_mes_rubricas.xlsx'

Limpeza e Pre-processamento dos dados para tentar identificar o caso a seguir como um Anomaly Detection
![image.png](attachment:1cf54f27-e734-4093-a92f-766002dc7ed0.png)

In [12]:
registration_df = processed_data(brute_file_path, "dados_12_meses_cadastro_e_std")

Processed data shapes:
Registration data: (151, 22)


C:\Users\joaoc\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Isolation Forest - Rubricas Detalhadas

In [14]:
resultado_rubricas = []

dados = registration_df[registration_df['cpf_servidor'] == 35042770763]
dados = dados.fillna(0)
series = dados.drop('cpf_servidor', axis=1)
series = series.loc[:, (series != 0).any(axis=0)]
        


In [15]:
series = series.set_index('mes')

In [16]:
resultado_rubricas = []
window = series
while len(window) > 10:
    # Define contaminação de forma mais eficiente
    contamination = 1/len(series) if len(series) < 20 else 1/20
    
    # Cria e ajusta o modelo
    iso_forest = IsolationForest(
        contamination=contamination,
        random_state=42,
        n_jobs=-1  # Paraleliza o processamento
    )
    
    # Faz a predição
    anomalias = iso_forest.fit_predict(window)
    
    # Verifica anomalias de forma vetorizada
    if anomalias[-1] == -1 and window.loc[window.index[-1], 'DECIMO TERCEIRO'] == 0: 
        resultado_rubricas.append({
            "cpf_servidor": dados['cpf_servidor'].iloc[0],
            "mes": window.index[-1]
        })
        
    # Exclui o ultimo mes, para verificar anomalia nos meses anteriores
    window = window.iloc[:-1, :]

In [46]:
resultado_rubricas

[{'cpf_servidor': 35042770763, 'mes': Timestamp('2023-04-01 00:00:00')},
 {'cpf_servidor': 35042770763, 'mes': Timestamp('2022-04-01 00:00:00')},
 {'cpf_servidor': 35042770763, 'mes': Timestamp('2022-03-01 00:00:00')}]

## Isolation Forest - Remuneracao Total

In [19]:
resultado_remun = []

series_remun = dados.drop('cpf_servidor', axis=1)
series_remun = series_remun.loc[:, (series_remun != 0).any(axis=0)]
series_remun['redim'] = series_remun.iloc[:, 1:].sum(axis=1)
series_remun = series_remun.set_index('mes')
series_remun = series_remun.iloc[:, -1:]


In [20]:
window = series_remun
while len(window) > 10:
    # Define contaminação de forma mais eficiente
    contamination = 1/len(series) if len(series) < 20 else 1/20
    
    # Cria e ajusta o modelo
    iso_forest = IsolationForest(
        contamination=contamination,
        random_state=42,
        n_jobs=-1  # Paraleliza o processamento
    )
    
    # Faz a predição
    anomalias = iso_forest.fit_predict(window)
    
    # Verifica anomalias de forma vetorizada
    if anomalias[-1] == -1:  # Verifica apenas a última observação
        resultado_remun.append({
            "cpf_servidor": dados['cpf_servidor'].iloc[0],
            "mes": window.index[-1]
        })
        
    # Exclui o ultimo mes, para verificar anomalia nos meses anteriores
    window = window.iloc[:-1, :]


In [21]:
resultado_remun

[{'cpf_servidor': 35042770763, 'mes': Timestamp('2023-06-01 00:00:00')},
 {'cpf_servidor': 35042770763, 'mes': Timestamp('2022-11-01 00:00:00')},
 {'cpf_servidor': 35042770763, 'mes': Timestamp('2022-06-01 00:00:00')},
 {'cpf_servidor': 35042770763, 'mes': Timestamp('2022-04-01 00:00:00')},
 {'cpf_servidor': 35042770763, 'mes': Timestamp('2022-03-01 00:00:00')}]